<a href="https://colab.research.google.com/github/anaumghori/dreambooth-style-training/blob/main/Dreambooth_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP-1

!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install .
!pip install -r examples/dreambooth/requirements.txt
!pip install bitsandbytes --extra-index-url https://jllllll.github.io/bitsandbytes-wheels/
!pip install accelerate

In [ ]:
# STEP-2

from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
# STEP-3

%env MODEL_NAME=runwayml/stable-diffusion-v1-5
%env INSTANCE_DIR=/content/drive/MyDrive/dreambooth_dataset
%env OUTPUT_DIR=/content/drive/MyDrive/dreambooth_model
%env INSTANCE_PROMPT="foxy"
%cd examples/dreambooth

In [ ]:
# STEP-4

!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="$INSTANCE_PROMPT" \
  --resolution=512 \
  --train_batch_size=2 \
  --gradient_accumulation_steps=4 \
  --learning_rate=3e-7 \
  --max_train_steps=6000 \
  --mixed_precision="fp16" \
  --lr_scheduler="constant" \
  --gradient_checkpointing \
  --use_8bit_adam \
  --sample_batch_size=8 \
  --checkpointing_steps=100 \
  --validation_prompt="foxy" \
  --validation_steps=100 \
  --train_text_encoder

# **Resuming Training and Managing Checkpoints**
If your training is interrupted due to an error, you can resume from the last saved checkpoint without starting over. To do this:

* Copy the same code block you used for training in Step 4.
* Add the following line after ```--train_text_encoder```, replacing the path with the location of your last saved checkpoint:

    ```--resume_from_checkpoint=/content/drive/MyDrive/dreambooth_model/checkpoint-200```

If you’re saving models and checkpoints directly to Google Drive, you might encounter storage issues if Drive becomes full. One solution is to delete older checkpoints you no longer need. However, keep in mind:

* Deleting older checkpoints is irreversible, so only delete them if you’re sure you won’t need to recover those versions.
* Alternatively, you can download and store important checkpoints locally or on another storage service to free up Drive space while keeping backups.




In [ ]:
# STEP-5

from diffusers import DiffusionPipeline
import torch

model_path = "/content/drive/MyDrive/dreambooth_model"
pipeline = DiffusionPipeline.from_pretrained(
    model_path,
    torch_dtype=torch.float16
).to("cuda")

In [ ]:
# TEST THE MODEL BY GENERATING IMAGES

prompt = "A foxy character sitting down"
image = pipeline(prompt, num_inference_steps=50, guidance_scale=9.5).images[0]
image.save("/content/foxy.png")